In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd data_2023

/content/data_2023


In [ ]:
#압축파일 풀 곳
!mkdir data_2023

#데이터있는 주소
!unzip -qq drive/MyDrive/open.zip -d./data_2023

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt

from skimage.segmentation import clear_border
import skimage.morphology as mp
import scipy.ndimage.morphology as sm

In [ ]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

csv파일 설정

In [ ]:
submit1=pd.read_csv('kfold_try1_ford1.csv') #csv파일 이름
submit2=pd.read_csv('kfold_try1_ford2.csv')
submit3=pd.read_csv('kfold_try1_ford3.csv')
submit4=pd.read_csv('kfold_try1_ford4.csv')
submit5=pd.read_csv('kfold_try1_ford5.csv')

In [ ]:
result_total = []
for i in range(len(submit1)):
        result = rle_decode(submit1['mask_rle'][i], (224,224))
        result += rle_decode(submit2['mask_rle'][i], (224,224))
        result += rle_decode(submit3['mask_rle'][i], (224,224))
        result += rle_decode(submit4['mask_rle'][i], (224,224))
        result += rle_decode(submit5['mask_rle'][i], (224,224))
        result[result==1] = 0
        result[result==2] = 0
        result[result==3] = 1
        result[result==4] = 1
        result[result==5] = 1
        result = rle_encode(result)
        if result == '':
           result_total.append(-1)
        else:
           result_total.append(result)

In [ ]:
submit = pd.read_csv('sample_submission.csv')
submit['mask_rle'] = result_total
submit.to_csv('../drive/MyDrive/kfold_try1_hard_voting_5.csv', index=False) #이름변경

출력

In [ ]:
#test_data = pd.read_csv('test.csv')
submit_total=pd.read_csv('kfold_try1_hard_voting_5.csv')

for i in range(50):
    #test_image_path = test_data['img_path'][i]
    #test_image = cv2.imread(test_image_path)
    #test_mask1 = rle_decode(submit1['mask_rle'][i], (224,224))
    #test_mask2 = rle_decode(submit2['mask_rle'][i], (224,224))
    #test_mask3 = rle_decode(submit3['mask_rle'][i], (224,224))
    test_mask4 = rle_decode(submit_total['mask_rle'][i], (224,224))

    plt.figure(figsize=(2,2))

    '''
    plt.subplot(151)
    plt.imshow(test_image)
    plt.axis("off")
    plt.subplot(152)
    plt.imshow(test_mask1)
    plt.axis("off")
    plt.subplot(153)
    plt.imshow(test_mask2)
    plt.axis("off")
    plt.subplot(154)
    plt.imshow(test_mask3)
    plt.axis("off")
    '''
    plt.imshow(test_mask4)
    plt.axis("off")